In [2]:
import os

In [1]:
from pathlib import Path

class DisplayablePath(object):
    display_filename_prefix_middle = '├──'
    display_filename_prefix_last = '└──'
    display_parent_prefix_middle = '    '
    display_parent_prefix_last = '│   '

    def __init__(self, path, parent_path, is_last):
        self.path = Path(str(path))
        self.parent = parent_path
        self.is_last = is_last
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    @classmethod
    def make_tree(cls, root, parent=None, is_last=False, criteria=None):
        root = Path(str(root))
        criteria = criteria or cls._default_criteria

        displayable_root = cls(root, parent, is_last)
        yield displayable_root

        children = sorted(list(path
                               for path in root.iterdir()
                               if criteria(path)),
                          key=lambda s: str(s).lower())
        count = 1
        for path in children:
            is_last = count == len(children)
            if path.is_dir():
                yield from cls.make_tree(path,
                                         parent=displayable_root,
                                         is_last=is_last,
                                         criteria=criteria)
            else:
                yield cls(path, displayable_root, is_last)
            count += 1

    @classmethod
    def _default_criteria(cls, path):
        return True

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    def displayable(self):
        if self.parent is None:
            return self.displayname

        _filename_prefix = (self.display_filename_prefix_last
                            if self.is_last
                            else self.display_filename_prefix_middle)

        parts = ['{!s} {!s}'.format(_filename_prefix,
                                    self.displayname)]

        parent = self.parent
        while parent and parent.parent is not None:
            parts.append(self.display_parent_prefix_middle
                         if parent.is_last
                         else self.display_parent_prefix_last)
            parent = parent.parent

        return ''.join(reversed(parts))

In [5]:
p = os.getcwd()

'C:\\Users\\irina\\Ironhack\\dataV2-labs\\module-3\\RecSys-Spotify-Million-Playlists'

In [9]:
ignore = ['data-processed',  'dataset', 'Kaggle-RecSys-101', '.ipynb_checkpoints']
paths = DisplayablePath.make_tree(
    Path(p), 
    criteria=lambda path: True if path.name not in (ignore) else False
)
for path in paths:
    print(path.displayable())

RecSys-Spotify-Million-Playlists/
├── .gitignore
├── APPROACHES.md
├── credentials/
│   └── credentials.ipynb
├── data-processing-notebooks/
│   ├── collaborative-filtering-data-preparation-scale-down.ipynb
│   ├── collaborative-filtering-data-preparation.ipynb
│   ├── dictionaries_tracks_desc_pids.ipynb
│   ├── explore-with-provided-scripts.ipynb
│   ├── mpd-slice-combine.ipynb
│   ├── spotify-track-api.ipynb
│   └── track-uri-combine.ipynb
├── evaluation/
│   ├── ALS_binary_topn_500_100.csv
│   ├── ALS_binary_topn_500_10000.csv
│   ├── ALS_pos_topn_500_10000.csv
│   ├── CB_audio_features_1.csv
│   ├── CB_audio_features_10.csv
│   ├── CB_audio_features_1000.csv
│   ├── SVD_binary100_10000.csv
│   ├── SVD_binary50_10000.csv
│   ├── SVD_pos100_10000.csv
│   └── SVD_pos50_10000.csv
├── Make tree.ipynb
├── modeling-notebooks/
│   ├── CB00_Audio_features_full_data.ipynb
│   ├── CF00_Model_Surprise_50pct_sample.ipynb
│   ├── CF00_Model_Surprise_scaled_down.ipynb
│   ├── CF01_Memory_based_sc